In [35]:
from datasets import load_dataset

dataset = load_dataset("tatoeba", lang1="en", lang2="fr")
#dataset = load_dataset("Helsinki-NLP/opus_books", "en-fr")

In [36]:
print(dataset["train"][0])

{'id': '0', 'translation': {'en': 'When he asked who had broken the window, all the boys put on an air of innocence.', 'fr': "Lorsqu'il a demandé qui avait cassé la fenêtre, tous les garçons ont pris un air innocent."}}


In [30]:
# Requires transformers>=4.51.0
# Requires sentence-transformers>=2.7.0

from sentence_transformers import SentenceTransformer

# Load the model
embedding_model = SentenceTransformer("Qwen/Qwen3-Embedding-0.6B")

# We recommend enabling flash_attention_2 for better acceleration and memory saving,
# together with setting `padding_side` to "left":
# model = SentenceTransformer(
#     "Qwen/Qwen3-Embedding-0.6B",
#     model_kwargs={"attn_implementation": "flash_attention_2", "device_map": "auto"},
#     tokenizer_kwargs={"padding_side": "left"},
# )

# The queries and documents to embed
queries = [
    dataset["train"][5*j]["translation"]["en"] for j in range(5) 
]
documents = [
    dataset["train"][5*j]["translation"]["fr"] for j in range(5) 
]

# Encode the queries and documents. Note that queries benefit from using a prompt
# Here we use the prompt called "query" stored under `model.prompts`, but you can
# also pass your own prompt via the `prompt` argument
query_embeddings = embedding_model.encode(queries, prompt_name="query")
document_embeddings = embedding_model.encode(documents)

# Compute the (cosine) similarity between the query and document embeddings
similarity = embedding_model.similarity(query_embeddings, document_embeddings)
print(similarity)

tensor([[ 0.2660,  0.1054, -0.0056,  0.0786,  0.1598],
        [ 0.2711,  0.7198,  0.1266,  0.1034,  0.2096],
        [ 0.2078,  0.0960,  0.5121,  0.3228,  0.3134],
        [ 0.2341,  0.1815,  0.2131,  0.5805,  0.1967],
        [ 0.3629,  0.1869,  0.2967,  0.1742,  0.7120]])


In [39]:
from transformers import AutoModelForCausalLM, AutoTokenizer

checkpoint = "HuggingFaceTB/SmolLM2-360M-Instruct"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint)

# prepare the model input
prompt = """Translate the following sentence from English to French: 
'When he asked who had broken the window, all the boys put on an air of innocence.'
Output the translation and only the translation."""
messages = [
    {"role": "user", "content": prompt}
]
input_text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
)
model_inputs = tokenizer([input_text], return_tensors="pt").to(model.device)

# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=150
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 
output = tokenizer.decode(output_ids, skip_special_tokens=True).strip("\n")
print(output)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


assistant
'Quand il aimez quelqu'un, tout les jours les jongles ont un air de naivete.'


In [41]:

from sentence_transformers import SentenceTransformer

# Load the model
embedding_model = SentenceTransformer("Qwen/Qwen3-Embedding-0.6B")
queries = [
    dataset["train"][0]["translation"]["en"]
]
documents = [
    output
]
query_embeddings = embedding_model.encode(queries, prompt_name="query")
document_embeddings = embedding_model.encode(documents)

# Compute the (cosine) similarity between the query and document embeddings
similarity = embedding_model.similarity(query_embeddings, document_embeddings)
print(similarity)

tensor([[0.3731]])
